## Введение

В данной статье я реализую проект электронных ключей для автомобиля, которые позволяют открывать или закрывать машину в зависимости от нажатой кнопки. Проект будет несложным, но весьма полезным. В качестве пульта управления будет использоваться мобильный телефон, чтобы избежать необходимости применения двух микроконтроллеров.

Следует отметить, что программная аутентификация, авторизация, реализация электрического замка и другие элементы безопасности выходят за рамки данного проекта. Здесь я сосредоточусь на аппаратной части. Этот проект также можно адаптировать для любых ситуаций, где требуется управление дверями или устройствами с помощью телефона или устройства, оснащенного Bluetooth, например, для управления дверями дома, шкафчиками, сейфами и т. д.
Используемые устройства

Проект базируется на микроконтроллере ATmega328P, используемом в платах Arduino. Я выбрал Arduino Nano из-за её компактности и достаточности для управления одним внешним устройством.

Для связи с телефоном я использую модуль Bluetooth HC-05. Он прост в использовании, широко доступен и уже хорошо мне знаком. Однако вы можете выбрать другой модуль Bluetooth, подходящий под ваши требования. Адаптация кода для другой архитектуры не потребует больших усилий.

## HC-05

Модуль HC-05 удобен тем, что его допустимое напряжение питания (VCC) совпадает с напряжением Arduino. Для связи используется интерфейс UART, что значительно упрощает подключение. Единственное, потребуется делитель напряжения для линии RX, чтобы снизить 5 В до 3,3 В.
## Распиновка модуля:

    VCC: питание (3.6–6 В).
    GND: общий провод.
    TXD: передача данных.
    RXD: приём данных (нужен делитель напряжения для работы с Arduino).
    EN (или KEY): управляет режимами. При подтягивании к 3.3 В модуль переходит в режим настройки AT-команд.

## План подключения и разработки

    HC-05 VCC → Arduino VCC (5V).
    HC-05 GND → Arduino GND.
    HC-05 TX → Arduino RX (D0).
    HC-05 RX → Arduino TX (D1) через делитель напряжения (с 5 В до 3.3 В).
    Транзистор (биполярный) базой подключается к D3.

## Принцип работы транзистора:

Транзистор управляет состоянием электрической цепи замка. Когда через D3 подаётся сигнал, транзистор открывается, и двери автомобиля открываются. В этом состоянии автомобиль переходит в режим готовности: включается двигатель, обогрев сидений и другие системы. Когда сигнал на D3 прекращается, замок закрывается, и питание систем отключается.


## Написание кода 
Давайте напишем для начала базовый код, с проверкой состояния каждый миг:

```c

volatile char command; // Переменная для хранения команды

void setup() {
  Serial.begin(9600); 
  pinMode(2, INPUT); // D2 для INT0
  pinMode(3, OUTPUT); // D3 для управления
  
  EIMSK |= _BV(INT0); // Включить прерывание INT0
  EICRA |= _BV(ISC01); // Настроить прерывание на задний фронт (лог. 1 -> лог. 0)
  sei(); // Включить глобальные прерывания
}

ISR(INT0_vect) {
  if (Serial.available()) { // Проверяем, есть ли данные в буфере
    command = Serial.read(); // Читаем команду
  }
}

void loop() {
  if (command == '1') { // Включить светодиод
    digitalWrite(3, HIGH);
    command = 0; // Сбрасываем команду
  } else if (command == '0') { // Выключить светодиод
    digitalWrite(3, LOW);
    command = 0; // Сбрасываем команду
  }
}

Но такой подход несет очень большую и ненужную нагрузку на микроконтроллер. И чтобы это избежать я и подсоединил D1 к D2, дабы генерировалось прерывание и мне не нужно было постоянно проверять состояние пина. 
Так что давайте реализуем способ получше:


```c
void setup() {
 Serial.begin(9600); 
 pinMode(D2, INPUT);
 pinMode(D3, OUTPUT); 

 EIMSK |= (1 << INT0);
 EICRA = (1 << ISCO1); 

 sei();
} 

char command; 

ISR(INT0_vect) {
 command = Serial.read();
 if(command == ‘1’){
  digitalWrite(D3, HIGH); 
 }
 else if(command == ‘0’) {
  digitalWrite(D3, LOW); 
 }
}

void loop(){
 // пусто :)
}


Такой код хороший, но у него есть недостаток - прерывания должны работать быстро и четко. А если мы из прерывание полезем во внешние ресурсы - это будет не совсем правильно и может привести к сбоям. 
Поэтому давайте реализуем немного по другому: 

```c
void setup() {
 Serial.begin(9600); 
 pinMode(D2, INPUT);
 pinMode(D3, OUTPUT); 

 EIMSK |= (1 << INT0);
 EICRA = (1 << ISCO1); 

 sei();
 int locksState = 0;
} 


ISR(INT0_vect) {
 if(locksState == 1){
  digitalWrite(D3, HIGH); 
  locksState = 0;
 }
 else if(locksState == 0) {
  digitalWrite(D3, LOW); 
  locksState = 1;
 }
}

void loop(){
 // пусто :)
}


Мы отказались от проверки типа сигнала — открытие это или закрытие. В результате любое нажатие кнопки, независимо от её назначения, приведёт к смене состояния замков. Более того, внешние помехи также могут влиять на работу системы, что совершенно недопустимо в реальных условиях. Учитывая эти факторы, из трёх возможных подходов я бы выбрал второй, поскольку он предоставляет баланс между простотой реализации и устойчивостью к сбоям.

## Заключение

Этот проект демонстрирует, как можно использовать простые аппаратные и программные решения для создания базовой системы управления замком через Bluetooth. Несмотря на свою простоту, система обладает большим потенциалом для применения: от автомобильных замков до домашней автоматизации.

Однако, чтобы использовать её в реальных условиях, необходимо учитывать ряд нюансов: устойчивость к внешним помехам, безопасность передачи данных, энергоэффективность и надёжность компонентов. Эти аспекты выходят за рамки данного проекта, но они являются важным шагом к созданию более сложных и надёжных решений.

Я надеюсь, что данная работа будет полезна как начинающим разработчикам, так и тем, кто ищет идеи для реализации собственных проектов. 